# Basic Data Analysis

In [3]:
# import modules

import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.discrete.discrete_model import Logit
import matplotlib.pyplot as plt

## Berkson's paradox simulation

In [ ]:
def generate_data(p1, p2, N):

    binary_outcomes = [0, 1]
    d1 = np.random.choice(binary_outcomes, size=N, p=[1-p1, p1])
    d2 = np.random.choice(binary_outcomes, size=N, p=[1-p2, p2])
    return d1, d2

In [ ]:
d1, d2 = generate_data(0.1, 0.1, 100)
d1, d2

In [ ]:
zipped = [z for z in zip(d1, d2) if (z[0] + z[1] >= 1)]
d1_hosp, d2_hosp = tuple(zip(*zipped))
d1_hosp, d2_hosp

## Coin-flip experiments

In [ ]:
# function to compute logistic regression

def logitpval(x, y):
    
    x = np.array([np.ones_like(x), x])
    y = np.array(y)
    logit = Logit(y, x.T)
    res = logit.fit_regularized(alpha=0.1, disp=0)
    return res

In [ ]:
# function to generate random binary vectors

def generate_data(p1, p2, N):

    binary_outcomes = [0, 1]
    d1 = np.random.choice(binary_outcomes, size=N, p=[1-p1, p1])
    d2 = np.random.choice(binary_outcomes, size=N, p=[1-p2, p2])
    return d1, d2

In [ ]:
# generate random binary vectors (single step)
# with prescribed prevalences

d1, d2 = generate_data(0.1, 0.1, 100)

In [ ]:
# carry out the logistic regression p-val
# p-value and log-odds ratio

res = logitpval(d1, d2)
pval, lor = res.pvalues[1], res.params[1]
pval, lor

In [ ]:
# carry out many simulations generate many results

n_sim = 100
y, p = [], []
for _ in range(n_sim):
    d1, d2 = generate_data(0.1, 0.1, 1000)
    res = logitpval(d1, d2)
    p.append(res.pvalues[1])
    y.append(res.params[1])

In [ ]:
# distribution of log-odds ratios for many simulations

def jitter(x, sigma=0.01):
    
    eps = np.random.normal(0, sigma, size=len(x))
    return x + eps

plt.boxplot(y, showfliers=False)
plt.scatter(jitter(np.ones(len(y))), y, alpha=0.5)
plt.ylabel('log OR')
plt.xticks([])
plt.title(f'Boxplot: {n_sim} simulations\n p1=p2=0.1 N=1000')
plt.savefig('./boxplot_full_population.png', dpi=200)
plt.show()

In [ ]:
# now fit the logistic regression with
# restricted vectors, i.e.,
# by selecting those samples with at least one hit

y, p = [], []
for _ in range(100):
    d1, d2 = generate_data(0.1, 0.1, 1000)
    zipped = [z for z in zip(d1, d2) if (z[0] + z[1] >= 1)]
    d1, d2 = tuple(map(np.array, zip(*zipped)))
    res = logitpval(d1, d2)
    p.append(res.pvalues[1])
    y.append(res.params[1])

In [ ]:
plt.boxplot(y, showfliers=False)
plt.scatter(jitter(np.ones(len(y))), y, alpha=0.5)
plt.ylabel('log OR')
plt.xticks([])
plt.title(f'Boxplot: {n_sim} simulations: Restricted Set\n p1=p2=0.1 N=1000')
plt.savefig('./boxplot_selected_population.png', dpi=200)
plt.show()

In [ ]:
# let's have a look at the last pair of
# binary vectors that has been generated
# it may seem as if the diseases repel each other...

d1, d2

## Tea testing challenge a.k.a. Fisher test

In [ ]:
from scipy.stats import hypergeom

[M, n, N] = [40, 20, 20]  # M: total cups of tea
                          # N: true milk-first cups of tea
                          # n: cups of tea chosen at the experiment

rv = hypergeom(M, n, N)
x = np.arange(0, n+1)
pmf_successes = rv.pmf(x)  # vector of probability masses for each
                           # possible outcome of the experiment

In [ ]:
# plot the discrete distribution of all possible outcomes
# for the lady tea challenge

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x, pmf_successes, 'bo')
ax.vlines(x, 0, pmf_successes, lw=2)
ax.set_xticks(x)
ax.set_xlabel('a = # correctly predicted milk-first cups of tea')
ax.set_ylabel('Probability of Contingency Table')
plt.savefig('./lady_tasting.png', dpi=200)
plt.show()

In [ ]:
# what is the probability mass comprising
# the outcomes at least as good as k = 12 successes?

sum(pmf_successes[12:21])

## Multiple test correction

In [ ]:
# Benjamini-Hochberg FDR

from statsmodels.stats.multitest import multipletests

pval_collection = [0.04, 0.15, 0.9, 0.01, 0.64, 0.07, 0.15, 0.64, 0.3]
qval_collection = multipletests(pval_collection, method='fdr_bh')[1]
qval_collection

# Supplementary Material: my first regression survival kit in Python

## Linear regression

In [ ]:
from statsmodels.regression.linear_model import OLS

# carry out linear regression on a synthetic dataset
# with known dependencies between covariate and response
# plus some noise

x = np.random.normal(20, 5, size=50)
y = 0.1 * x + np.random.normal(0, 1., size=50)
x = np.array([x, np.ones(len(x))])
model = OLS(y, x.T)
res = model.fit()
res.summary()

In [ ]:
# plot the data

plt.scatter(x[0,:], y, alpha=0.7)
plt.xlabel('temperature')
plt.ylabel('growth rate percentage')
plt.savefig('./linear_regression.png', dpi=200)
plt.show()

In [ ]:
# plot the data
# with fitting line

X = np.linspace(5, 35, num=100)
Y = res.params[0] * X + res.params[1]
plt.plot(X, Y, 'r--')
plt.scatter(x[0,:], y, alpha=0.7)
plt.xlabel('temperature')
plt.ylabel('growth rate percentage')
plt.savefig('./linear_regression_line.png', dpi=200)
plt.show()

In [ ]:
# carry out multivariable linear regression
# on synthetic dataset with known dependencies

from statsmodels.regression.linear_model import OLS

x = np.random.normal(20, 5, size=50)
pH = np.random.normal(7, 2, size=50)
p = np.random.normal(1, 0.1, size=50)
y = 0.1 * x + pH + 2.5 * p + np.random.normal(0, 1., size=50)
x = np.array([x, pH, p, np.ones(len(x))])
model = OLS(y, x.T)
res = model.fit()
res.summary()

## Logistic Regression

In [ ]:
# generate synthetic students data

shape, scale = 30, 1
hours = np.random.gamma(shape=shape, scale=scale, size=100)

# true params of the model

a = 1 / 5
b = -6

# synthetic probabilities

probs = 1 / (1 + np.exp((-1) * (hours * a + b)))
outcome = np.random.binomial(1, size=len(probs), p=probs)

In [ ]:
# plot data

plt.scatter(hours, outcome, alpha=0.6)
plt.yticks([0, 1], ['fail', 'pass'])
plt.xlabel('hours of study')
plt.savefig('./logistic.png', dpi=200)
plt.title('Test Results')
plt.show()

In [ ]:
# plot data with true logistic fitting curve

x = np.linspace(10, 50, num=100)
probs = 1 / (1 + np.exp((-1) * (x * a + b)))
plt.plot(x, probs, 'black')
plt.scatter(hours, outcome, alpha=0.6)
plt.xlabel('hours of study')
plt.ylabel('probability to pass')
plt.savefig('./logistic_line.png', dpi=200)
plt.title('Test Results')
plt.show()

In [ ]:
# carry out logistic regression

x = np.array([np.ones_like(hours), hours])
y = np.array(outcome)
logit = Logit(y, x.T)
res = logit.fit()
res.summary()

In [ ]:
# lady tea challenge revisited
# carry out regression with logistic regression

y = np.array([1, 1, 1, 1, 0, 0, 0, 0])
x = np.array([1, 1, 1, 0, 1, 0, 0, 0])
x = np.array([x, np.ones(len(x))])
logit = Logit(y, x.T)
res = logit.fit()
res.summary()